In [ ]:
##################
# 프로그램명: 학생 성적관리 프로그램 (SQLite 연동)
# 작성자: 소프트웨어학부 2023078086 최은재
# 작성일: 2025-06-10
# 프로그램 설명:
"""
    학생의 세개의 교과목 (영어, C-언어, 파이썬)에 대하여
    SQLite 데이터베이스를 이용하여 학번, 이름, 영어점수, C-언어 점수, 파이썬 점수를 관리하고
    총점, 평균, 학점, 등수를 계산하는 프로그램

    - 데이터베이스 연동: 학생 정보 영구 저장
    - CRUD 기능: 삽입, 조회, 수정, 삭제
    - 성적 계산: 총점, 평균, 학점, 등수
    - 데이터 분석: 통계 정보 제공
"""
##################

import sqlite3
import os

# 학생 한 명의 정보를 저장하고, 계산하는 클래스
class Student:
    def __init__(self, stu_num, name, eng, c_lang, python):
        self.stu_num = stu_num
        self.name = name
        self.eng = eng
        self.c_lang = c_lang
        self.python = python
        self.total = 0
        self.average = 0.0
        self.grade = ''
        self.rank = 1

    def calculate_score(self):
        self.total = self.eng + self.c_lang + self.python
        self.average = round(self.total / 3, 2)

    def assign_grade(self):
        avg = self.average
        if avg >= 95:
            self.grade = 'A+'
        elif avg >= 90:
            self.grade = 'A'
        elif avg >= 85:
            self.grade = 'B+'
        elif avg >= 80:
            self.grade = 'B'
        elif avg >= 75:
            self.grade = 'C+'
        elif avg >= 70:
            self.grade = 'C'
        elif avg >= 65:
            self.grade = 'D+'
        elif avg >= 60:
            self.grade = 'D'
        else:
            self.grade = 'F'

    def print_info(self):
        print(f"{self.stu_num}\t{self.name}\t{self.eng}\t{self.c_lang}\t{self.python}\t"
              f"{self.total}\t{self.average}\t{self.grade}\t{self.rank}")

    def to_dict(self):
        return {
            'stu_num': self.stu_num,
            'name': self.name,
            'eng': self.eng,
            'c_lang': self.c_lang,
            'python': self.python,
            'total': self.total,
            'average': self.average,
            'grade': self.grade
        }


# 데이터베이스 관리 클래스
class DatabaseManager:
    def __init__(self, db_name='student_grades.db'):
        self.db_name = db_name
        self.init_database()

    def init_database(self):
        #데이터베이스 및 테이블 초기화
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                stu_num INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                eng INTEGER NOT NULL,
                c_lang INTEGER NOT NULL,
                python INTEGER NOT NULL,
                total INTEGER,
                average REAL,
                grade TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
        
    #학생 정보를 데이터베이스에 삽입
    def insert_student(self, student):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT INTO students (stu_num, name, eng, c_lang, python, total, average, grade)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (student.stu_num, student.name, student.eng, student.c_lang, 
                  student.python, student.total, student.average, student.grade))
            
            conn.commit()
            return True
        except sqlite3.IntegrityError:
            print(f"오류: 학번 {student.stu_num}은 이미 존재합니다.")
            return False
        finally:
            conn.close()
            
    #학번으로 학생 삭제
    def delete_student(self, stu_num):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('DELETE FROM students WHERE stu_num = ?', (stu_num,))
        
        if cursor.rowcount > 0:
            conn.commit()
            conn.close()
            return True
        else:
            conn.close()
            return False
            
    #학생 정보 수정
    def update_student(self, student):    
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('''
            UPDATE students 
            SET name=?, eng=?, c_lang=?, python=?, total=?, average=?, grade=?, 
                updated_at=CURRENT_TIMESTAMP
            WHERE stu_num=?
        ''', (student.name, student.eng, student.c_lang, student.python,
              student.total, student.average, student.grade, student.stu_num))
        
        if cursor.rowcount > 0:
            conn.commit()
            conn.close()
            return True
        else:
            conn.close()
            return False
            
    #학번으로 학생 조회
    def get_student_by_id(self, stu_num):  
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM students WHERE stu_num = ?', (stu_num,))
        row = cursor.fetchone()
        conn.close()
        
        if row:
            return Student(row[0], row[1], row[2], row[3], row[4])
        return None
        
    #이름으로 학생 조회
    def get_student_by_name(self, name):
        
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM students WHERE name LIKE ?', (f'%{name}%',))
        rows = cursor.fetchall()
        conn.close()
        
        students = []
        for row in rows:
            student = Student(row[0], row[1], row[2], row[3], row[4])
            student.total = row[5]
            student.average = row[6]
            student.grade = row[7]
            students.append(student)
        
        return students
        
    #모든 학생 조회
    def get_all_students(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM students ORDER BY stu_num')
        rows = cursor.fetchall()
        conn.close()
        
        students = []
        for row in rows:
            student = Student(row[0], row[1], row[2], row[3], row[4])
            student.total = row[5]
            student.average = row[6]
            student.grade = row[7]
            students.append(student)
        
        return students
        
    #총점 기준으로 정렬된 학생 조회
    def get_students_sorted_by_total(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM students ORDER BY total DESC')
        rows = cursor.fetchall()
        conn.close()
        
        students = []
        for row in rows:
            student = Student(row[0], row[1], row[2], row[3], row[4])
            student.total = row[5]
            student.average = row[6]
            student.grade = row[7]
            students.append(student)
        
        return students

    #평균 점수 이상인 학생 수 카운트
    def count_students_over_average(self, threshold=80):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT COUNT(*) FROM students WHERE average >= ?', (threshold,))
        count = cursor.fetchone()[0]
        conn.close()
        
        return count
        
    #전체 통계 정보
    def get_statistics(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT 
                COUNT(*) as total_students,
                AVG(average) as class_average,
                MAX(total) as highest_total,
                MIN(total) as lowest_total,
                AVG(eng) as avg_eng,
                AVG(c_lang) as avg_c_lang,
                AVG(python) as avg_python
            FROM students
        ''')
        
        stats = cursor.fetchone()
        conn.close()
        
        return {
            'total_students': stats[0],
            'class_average': round(stats[1], 2) if stats[1] else 0,
            'highest_total': stats[2] if stats[2] else 0,
            'lowest_total': stats[3] if stats[3] else 0,
            'avg_eng': round(stats[4], 2) if stats[4] else 0,
            'avg_c_lang': round(stats[5], 2) if stats[5] else 0,
            'avg_python': round(stats[6], 2) if stats[6] else 0
        }


# 성적 관리 클래스
class EnhancedGradeManager:
    def __init__(self):
        self.db = DatabaseManager()
        self.students = []
        
    #데이터베이스에서 모든 학생 로드
    def load_students_from_db(self):
        self.students = self.db.get_all_students()
        self.assign_ranks()

    #학생 추가
    def add_student(self):
        try:
            stu_num = int(input("학번: "))
            name = input("이름: ")
            eng = int(input("영어 점수 (0-100): "))
            c_lang = int(input("C-언어 점수 (0-100): "))
            python = int(input("파이썬 점수 (0-100): "))

            # 점수 범위 검사
            if not all(0 <= score <= 100 for score in [eng, c_lang, python]):
                print("점수는 0-100 사이여야 합니다.")
                return

            student = Student(stu_num, name, eng, c_lang, python)
            student.calculate_score()
            student.assign_grade()

            if self.db.insert_student(student):
                print(f"학생 {name} 추가 완료")
                self.load_students_from_db()
            
        except ValueError:
            print("올바른 숫자를 입력해주세요.")
            
    #학생 삭제
    def delete_student(self, stu_num):
        if self.db.delete_student(stu_num):
            print(f"학번 {stu_num} 학생 삭제 완료!")
            self.load_students_from_db()
        else:
            print("삭제할 학생을 찾을 수 없습니다.")

    def update_student(self, stu_num):
        #학생 정보 수정
        student = self.db.get_student_by_id(stu_num)
        if not student:
            print("해당 학번의 학생을 찾을 수 없습니다.")
            return

        print(f"현재 정보: {student.name}, 영어:{student.eng}, C언어:{student.c_lang}, 파이썬:{student.python}")
        
        try:
            name = input(f"이름 ({student.name}): ") or student.name
            eng = input(f"영어 점수 ({student.eng}): ")
            c_lang = input(f"C-언어 점수 ({student.c_lang}): ")
            python = input(f"파이썬 점수 ({student.python}): ")

            eng = int(eng) if eng else student.eng
            c_lang = int(c_lang) if c_lang else student.c_lang
            python = int(python) if python else student.python

            if not all(0 <= score <= 100 for score in [eng, c_lang, python]):
                print("점수는 0-100 사이여야 합니다.")
                return

            updated_student = Student(stu_num, name, eng, c_lang, python)
            updated_student.calculate_score()
            updated_student.assign_grade()

            if self.db.update_student(updated_student):
                print("학생 정보 수정 완료!")
                self.load_students_from_db()

        except ValueError:
            print("올바른 숫자를 입력해주세요.")
    
    #학번으로 검색
    def search_by_id(self, stu_num):
        student = self.db.get_student_by_id(stu_num)
        if student:
            student.calculate_score()
            student.assign_grade()
            print("\n검색 결과:")
            print("학번\t\t이름\t영어\tC-언어\t파이썬\t총점\t평균\t학점")
            print("=" * 70)
            student.print_info()
        else:
            print("해당 학번의 학생을 찾을 수 없습니다.")

    #이름으로 검색
    def search_by_name(self, name):
        students = self.db.get_student_by_name(name)
        if students:
            print(f"\n'{name}' 검색 결과:")
            print("학번\t\t이름\t영어\tC-언어\t파이썬\t총점\t평균\t학점")
            print("=" * 70)
            for student in students:
                student.print_info()
        else:
            print(f"'{name}'과 일치하는 학생을 찾을 수 없습니다.")

    #등수 계산
    def assign_ranks(self):
        for i in range(len(self.students)):
            self.students[i].rank = 1
            for j in range(len(self.students)):
                if self.students[i].average < self.students[j].average:
                    self.students[i].rank += 1
    #총점 기준 정렬
    def sort_by_total(self):

        self.students = self.db.get_students_sorted_by_total()
        self.assign_ranks()
        print("총점 기준으로 정렬 완료!")

    #전체 학생 출력
    def print_all(self):
        if not self.students:
            print("등록된 학생이 없습니다.")
            return

        print("\n\t\t\t성적 관리 프로그램 (SQLite 연동)")
        print("=" * 80)
        print("학번\t\t이름\t영어\tC-언어\t파이썬\t총점\t평균\t학점\t등수")
        print("=" * 80)
        
        for student in self.students:
            student.print_info()
        
        print("=" * 80)
        print(f"* 평균 점수 80점 이상인 학생 수: {self.db.count_students_over_average(80)}명")

    #통계 정보 출력
    def show_statistics(self):

        stats = self.db.get_statistics()
        
        print("\n성적 통계 정보")
        print("=" * 50)
        print(f"총 학생 수: {stats['total_students']}명")
        print(f"전체 평균: {stats['class_average']}점")
        print(f"최고 총점: {stats['highest_total']}점")
        print(f"최저 총점: {stats['lowest_total']}점")
        print(f"영어 평균: {stats['avg_eng']}점")
        print(f"C-언어 평균: {stats['avg_c_lang']}점")
        print(f"파이썬 평균: {stats['avg_python']}점")
        print("=" * 50)

    #데이터베이스 백업
    def backup_database(self):

        import shutil
        from datetime import datetime
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_name = f"student_grades_backup_{timestamp}.db"
        
        try:
            shutil.copy2(self.db.db_name, backup_name)
            print(f"데이터베이스 백업 완료: {backup_name}")
        except Exception as e:
            print(f"백업 실패: {e}")

#메인 메뉴 함수
def main_menu():
    gm = EnhancedGradeManager()
    gm.load_students_from_db()
    
    print("학생 성적 관리 시스템 (SQLite 연동)")
    print("데이터베이스 연결 완료!")
    
    while True:
        print("\n" + "="*60)
        print("메뉴 선택")
        print("="*60)
        print("1. 학생 추가")
        print("2. 학생 삭제")
        print("3. 학생 정보 수정")
        print("4. 학번으로 검색")
        print("5. 이름으로 검색")
        print("6. 총점 기준 정렬")
        print("7. 전체 학생 출력")
        print("8. 통계 정보 보기")
        print("9. 데이터베이스 백업")
        print("0. 프로그램 종료")
        print("="*60)
        
        choice = input("선택 (0-9): ").strip()
        
        if choice == '1':
            gm.add_student()
            
        elif choice == '2':
            try:
                stu_num = int(input("삭제할 학생의 학번: "))
                gm.delete_student(stu_num)
            except ValueError:
                print("올바른 학번을 입력해주세요.")
                
        elif choice == '3':
            try:
                stu_num = int(input("수정할 학생의 학번: "))
                gm.update_student(stu_num)
            except ValueError:
                print("올바른 학번을 입력해주세요.")
                
        elif choice == '4':
            try:
                stu_num = int(input("검색할 학번: "))
                gm.search_by_id(stu_num)
            except ValueError:
                print("올바른 학번을 입력해주세요.")
                
        elif choice == '5':
            name = input("검색할 이름 (부분 검색 가능): ").strip()
            if name:
                gm.search_by_name(name)
            else:
                print("이름을 입력해주세요.")
                
        elif choice == '6':
            gm.sort_by_total()
            
        elif choice == '7':
            gm.print_all()
            
        elif choice == '8':
            gm.show_statistics()
            
        elif choice == '9':
            gm.backup_database()
            
        elif choice == '0':
            print("프로그램을 종료합니다. 감사합니다!")
            break
            
        else:
            print("잘못된 선택입니다. 0-9 사이의 숫자를 입력해주세요.")


if __name__ == "__main__":
    main_menu()

학생 성적 관리 시스템 (SQLite 연동)
데이터베이스 연결 완료!

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료


선택 (0-9):  1
학번:  20231234
이름:  김짱구
영어 점수 (0-100):  80
C-언어 점수 (0-100):  90
파이썬 점수 (0-100):  85


학생 김짱구 추가 완료!

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료


선택 (0-9):  1
학번:  20231111
이름:  김철수
영어 점수 (0-100):  100
C-언어 점수 (0-100):  95
파이썬 점수 (0-100):  90


학생 김철수 추가 완료!

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료


...

총점 기준으로 정렬 완료!

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료


선택 (0-9):  7



			성적 관리 프로그램 (SQLite 연동)
학번		이름	영어	C-언어	파이썬	총점	평균	학점	등수
20231111	김철수	100	95	90	285	95.0	A+	1
20231004	김유리	90	95	95	280	93.33	A	2
20232222	김맹구	90	88	92	270	90.0	A	3
20233333	김훈이	88	87	90	265	88.33	B+	4
20231234	김짱구	80	90	85	255	85.0	B+	5
1234	김짱구	50	50	50	150	50.0	F	6
* 평균 점수 80점 이상인 학생 수: 5명

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료


선택 (0-9):  9


데이터베이스 백업 완료: student_grades_backup_20250610_125822.db

메뉴 선택
1. 학생 추가
2. 학생 삭제
3. 학생 정보 수정
4. 학번으로 검색
5. 이름으로 검색
6. 총점 기준 정렬
7. 전체 학생 출력
8. 통계 정보 보기
9. 데이터베이스 백업
0. 프로그램 종료
